<table width="100%">
<tr>
    <td align="left" width="50%" style="border: 0px; padding-left: 20px;">
        <img src="img/udc.png" width="300">
    </td>
    <td align="right" width="50%" style="border: 0px; padding-right: 50px;">
        <img src="img/logo_mint.svg" width="250">
    </td>    
</tr>
</table>

<center>
<h1 style="color: #d60e8c;">Robobo: Robot Físico</h1> 
</center>

<h2>Contenidos</h2>

<div class="alert alert-block alert-info" 
     style="margin-top: 20px; padding-top:10px; border: 1px solid #d60e8c; border-radius: 20px; background:transparent;">
    <ul>
        <li><a href="#intro">Introducción</a></li>
        <ul>
            <li><a href="#battery">readBatteryLevel(device)</a></li>
            <li><a href="#clap">readClapCounter()</a></li>
            <li><a href="#clapreset">resetClapCounter()</a></li>
            <li><a href="#clapwhen">whenClapDetected(callback)</a></li> 
        </ul>
        <li><a href="#ej">Ejercicios</a></li>
    </ul>
</div>

<a id="intro" style="text-decoration:none;">
<h1 style="color: #d60e8c; border-bottom: 2px solid #d60e8c;padding-bottom: 5px;">Introducción</h1>
</a>

Este cuaderno está dedicado exclusivamente al Robobo **real**, para conocer aquellos métodos que no se pueden ejecutar en el simulador. Por tanto, para probarlos, es necesario disponer de un robot real, un smartphone y una red wi-fi en la que se puedan conectar el smartphone y el ordenador. El proceso de configuración y conexión del robot real se explica en el cuaderno 1.

Recordad que en los programas nos debemos conectar a la IP del robot que cada uno tenga asignada en la app de Robobo en el smatphone. Por tanto, al ejecutar este código, **cambiad la IP por la vuestra**:

In [ ]:
#Escribir IP aquí
IP_ROBOBO="192.168.1.140"

<a id="battery" style="text-decoration:none;">
<h3 style="color: #d60e8c;">readBatteryLevel(device)</h2>
</a>


**Función:** Lee el nivel de batería de la base o del smartphone.

**Parámetros:**
- device (string): Dispositivo del que se desea leer la batería. Puede ser ‘base’ para la base o
'phone' para el smartphone.

**Devuelve:**
- El nivel de batería (int). Los valores están entre 0 (vacía) y 100 (llena).

Para probar este método, con el robot real preparado, ejecutad el ejemplo de abajo en el cual Robobo dirá primero el nivel de batería de la base, y después el del teléfono

In [ ]:
from robobopy.Robobo import Robobo
rob = Robobo(IP_ROBOBO)
rob.connect()

#Retraso para darle margen y poder leer la batería sin problema
rob.wait(1)
#Robobo dice el nivel de batería de la base y el del teléfono
rob.sayText("El nivel de batería de la base es " + str(rob.readBatteryLevel('base')))
rob.sayText("El nivel de batería del teléfono es " + str(rob.readBatteryLevel('phone')))

Este método es de gran utilidad en robótica autónoma, ya que los robots pueden operar por largos periodos de tiempo, y debemos asegurar que su batería no se termina abruptamente. Por ejemplo, se podría utilizar este método para leer el nivel de batería de la base y el smartphone periódicamente, y al bajar de un valor de seguridad, se avisaría al usuario por voz para que recargue el robot.

<a id="#clap" style="text-decoration:none;">
<h3 style="color: #d60e8c;">readClapCounter()</h2>
</a>

**Función**: Lee el número de palmadas registrado desde el último reinicio.

**Devuelve:**
- El número de palmadas (int).

<a id="#clapreset" style="text-decoration:none;">
<h3 style="color: #d60e8c;">resetClapCounter()</h2>
</a>


**Función:** Reinicia el contador de palmadas.

En el ejemplo de abajo se hará una demostración de los dos métodos sobre la detección de palmadas:

- Cada 2 segundos el programa comprueba cuantas palmadas se hicieron en ese intervalo.
- Si se hicieron como mínimo 2 palmadas, Robobo pondrá cara de sorprendido y emitirá un sonido de risa. De lo contrario, pondrá una expresión normal.
- En ambos casos se reiniciará el contador.
- El programa dura 1 minuto aproximadamente

In [ ]:
from robobopy.Robobo import Robobo
from robobopy.utils.Sounds import Sounds
from robobopy.utils.Emotions import Emotions

#Conexión con el Robobo real
rob = Robobo(IP_ROBOBO)
rob.connect()

#Variable del tiempo de ejecución
timeout=0
while timeout<30:
    #Durante estos dos segundos el estudiante debe dar las palmadas que pueda
    rob.wait(2)
    print(f"{rob.readClapCounter()} palmadas en 2 segundos")
    #Si dió más de 1 palmada, Robobo reaccionará
    if rob.readClapCounter() > 1:
        rob.playSound(Sounds.LAUGH)
        rob.setEmotionTo(Emotions.SURPRISED)
        rob.wait(0.25)
    else:
    #En caso de que no sean suficientes, volverá a su estado normal
        rob.setEmotionTo(Emotions.NORMAL)
    #Se reinicia el contador
    rob.resetClapCounter()
    timeout+=1

#Se desconecta y termina el programa
rob.disconnect()


<a id="#clapwhen" style="text-decoration:none;">
<h3 style="color: #d60e8c;">whenClapDetected(callback)</h2>
</a>

**Función**: Establece la función callback que se llama cuando se detecta una nueva palmada.

**Parámetros:**
- callback (fun): La función callback que será llamada.

Ejecutad el siguiente programa:

- Robobo estará moviendose a una velocidad de 10 constantemente.
- Si en cualquier momento alguien hace una palmada, Robobo se detendrá y dirá "Me has mandado parar". No volverá a moverse hasta que termine el programa.
- Cuando hayan pasado aproximadamente 25 segundos, se detendrán los motores si aún no oyó una palmada, y pondrá una cara de **dormido**. En ese momento habrá finalizado el programa.

In [ ]:
from robobopy.Robobo import Robobo
from robobopy.utils.Emotions import Emotions

#Función que se usará como callback. Detiene los motores completamente
def clap():
    rob.stopMotors()
    rob.sayText('Me has mandado parar')
rob = Robobo(IP_ROBOBO)
rob.connect()

#Robobo se mueve de forma constante
rob.moveWheels(10,10)

#Declaración de la función de interupción
rob.whenClapIsDetected(clap)

# El programa se mantiene en ejecución continua durante 25 segundos
tiempo=0
while tiempo<250:
    rob.wait(0.1)
    tiempo+=1

#Detiene los motores si no se hizo antes
rob.stopMotors()
rob.setEmotionTo(Emotions.SLEEPING)

#Desconexión del robot y finalización del programa
rob.disconnect()

<a id="ej" style="text-decoration:none;">
<h1 style="color: #d60e8c; border-bottom: 2px solid #d60e8c;padding-bottom: 5px;">Ejercicios</h1>
</a>

**EJERCICIO 1**

Realizar un programa en el que Robobo:

- Gire sobre su eje a una velocidad constante.
-  El valor de la velocidad dependerá de la batería de la base. La ecuación será: ***Velocidad de giro= 3 + Batería de Robobo/5***
- La expresión de Robobo debe cambiar en función de la batería del smarthphone. Si la batería es mayor que el 20%, su expresión debe ser normal. Si está entre el 10 y 20%, pondrá una expresión de cansado. Si es inferior al 10%, pondrá una expresión de dormido y los motores no se moverán
- El programa durará 1 minuto. Luego tendrá que detener los motores y finalizar el programa.


In [ ]:
#INSERTAR CÓDIGO AQUÍ

**EJERCICIO 2**

Realiza un programa en el que Robobo:
- Esté quieto y con cara de **dormido**.
- Si oye mínimo 3 palmadas, pondrá una cara **feliz** y avanzará a una velocidad constante de **10**
- Si vuelve a oir mínimo 3 palmadas, pondrá una cara de **dormido** y se detendrá
- El programa durará 1 minuto. Una vez acabe, si está "despiero", parará los motores y dirá "ESTOY CANSADO", y pondrá cara de dormido

In [ ]:
#INSERTAR CÓDIGO AQUÍ

**EJERCICIO 3**

Mediante objetos de colores, ya sean bloques o pelotas, realiza un programa en el que Robobo:

- Avance de forma constante a una velocidad de 10.
- Cuando vea un objeto de un color en concreto en los extremos derecho o izquierdo de la cámara, gire hacia un lado o hacia otro a una velocidad de 15. No girará sobre su eje, solo cambiará la velocidad de una rueda.
- Usar la función de interrupción de los blobs.
- El tamaño del objeto debe ser mínimo de 20 pixeles.
- El tiempo de ejecución del programa será de máximo 120 segundos.

In [ ]:
#INSERTAR CÓDIGO AQUÍ

**EJERCICIO 4**

Este ejercicio se hará en una habitación con una iluminación baja:

- Robobo girará hacia la derecha sobre su eje buscando un foco de luz. Para ello, estará leyendo la luminosidad del ambiente constantemente. Si detecta que el foco de luz es mayor que 20 lux se parará y se acercará al oco.
- Robobo avanzará recto hacia la luz hasta que la luminosidad leída sea de 40.
- Si el foco de luz desaparece, Robobo deberá buscar la luz de nuevo.
- El programa durará 1 minuto. Cuando termine, se detendrá.

*Los valores de las condiciones se pueden modificar dependiendo del entorno en el que se haga el ejercicio. Simplemente son orientativos*

In [ ]:
#INSERTAR CÓDIGO AQUÍ

**EJERCICIO 5**

Colocar el Robobo encima de una mesa y ejecutad un código en el que:

- Robobo vaya a una velocidad de 10 constantemente.
- Si Robobo está cerca de caerse y los sensores delanteros que **miran hacia abajo** dejan de detectar valores, Robobo girará sobre su eje hasta que ambos sensores vuelvan a recibir valores.
- En cualquier momento se podrá detener el Robobo si oye una palmada (Usar la función de interrupción de las palmadas). Y si 
vuelve a oir otra palmada, volverá a  moverse
- El programa durará aproximadamente 30 segundos. Cuando acabe, se pararán los motores y Robobo dirá "Se acabó el trabajo de hoy", y pondrá cara de **dormido**. En ese momento finalizará el programa.

In [ ]:
#INSERTAR CÓDIGO AQUÍ